## Imports

In [23]:
from typing import Callable, Any, Dict
import re

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing

from natsort import natsorted

## Global information about each suite

In [24]:
runs = {
    run: {"version": version.name}
    for version in Path("./out").iterdir()
    for run in version.iterdir()
}


def add_entry_to_run(
    new_key: str, f: Callable[[Path, Dict[str, Any]], Any]
) -> Dict[Path, Dict[str, Any]]:
    global runs
    for key, value in runs.items():
        value.update({new_key: f(key, value)})


add_entry_to_run("name", lambda p, d: p.name)

### Time

In [25]:
def get_time(p:Path, d):
    file_path = p.rglob("klee/info").__next__()
    if file_path.exists():
        with open(file_path, "r") as file:
            res = re.search(r"--max-time=(\w*)", file.read())
            if res:
                return res.group(1)
    else:
        print(f"Error for {p}")
        return None

add_entry_to_run("time", get_time)

## Per util information

### Setup

In [26]:
add_entry_to_run("df", lambda p, d: pd.DataFrame(columns=[e.name for e in p.iterdir() if e.is_dir()]))

def add_entry_for_utils(key: str, f: Callable[[Path], Any]) -> None:
    """
    Add entry for all utils

    Paramenters:
    key (str): key to add the new value at in the dataframe
    f (Callable[[Path], Any]): function taking the path to the subfolder for the util and returning the appropriate value
    """

    def adder(p: Path, d):
        df = d["df"]
        res = {}
        for util in df.columns:
            path = p / util
            if not path.exists():
                raise Exception(f"Path \"{util}\" does not exist")
            res[util] = f(path)
        df.loc[key] = res
        return df
    add_entry_to_run("df", adder)

### Number of errors according to KLEE

In [27]:
def read_num_errors(error_type:str) -> Callable[[Path], str]:
    def f(util_path: Path) -> str:
        file_path = util_path / "klee"
        if file_path.exists():
            return str(len(list(file_path.glob(f"*{error_type}.err"))))
        else:
            print(f"Error for {util_path}")
            return None
    return f
error_types = [file.name for run in runs.keys() for file in list(run.glob("**/*.err"))]
error_types = [e.split(".")[-2] for e in error_types]
error_types = list(set(error_types))
for error_type in error_types:
    add_entry_for_utils(f"num_errors ({error_type})", read_num_errors(error_type))
add_entry_for_utils(f"num_errors (total)", read_num_errors(""))

Error for out/coreutils-8.25/1h-3/hostname
Error for out/coreutils-8.25/1h-3/setuidgid
Error for out/coreutils-8.25/1h-2/hostname
Error for out/coreutils-8.25/1h-2/setuidgid
Error for out/coreutils-8.25/1h/hostname
Error for out/coreutils-8.25/1h/setuidgid
Error for out/coreutils-9.4/1h-3/hostname
Error for out/coreutils-9.4/1h-3/setuidgid
Error for out/coreutils-9.4/1h-2/hostname
Error for out/coreutils-9.4/1h-2/setuidgid
Error for out/coreutils-9.4/1h/hostname
Error for out/coreutils-9.4/1h/setuidgid
Error for out/coreutils-8.25/1h-3/hostname
Error for out/coreutils-8.25/1h-3/setuidgid
Error for out/coreutils-8.25/1h-2/hostname
Error for out/coreutils-8.25/1h-2/setuidgid
Error for out/coreutils-8.25/1h/hostname
Error for out/coreutils-8.25/1h/setuidgid
Error for out/coreutils-9.4/1h-3/hostname
Error for out/coreutils-9.4/1h-3/setuidgid
Error for out/coreutils-9.4/1h-2/hostname
Error for out/coreutils-9.4/1h-2/setuidgid
Error for out/coreutils-9.4/1h/hostname
Error for out/coreutils-9

### Coverage according to KLEE

In [28]:
def read_klee_csv(csv_name: str) -> Callable[[Path], str]:
    def f(util_path: Path) -> str:
        file_path = util_path / "klee-stats.csv"
        if file_path.exists() and file_path.stat().st_size > 0:
            df = pd.read_csv(file_path)
            return str(df[csv_name][0])
        else:
            print(f"Error for {csv_name} — {util_path}")
            return None
    return f

add_entry_for_utils("klee_ICov", read_klee_csv("ICov(%)"))
add_entry_for_utils("klee_BCov", read_klee_csv("BCov(%)"))

Error for ICov(%) — out/coreutils-8.25/1h-3/hostname
Error for ICov(%) — out/coreutils-8.25/1h-3/setuidgid
Error for ICov(%) — out/coreutils-8.25/1h-2/hostname
Error for ICov(%) — out/coreutils-8.25/1h-2/setuidgid
Error for ICov(%) — out/coreutils-8.25/1h/hostname
Error for ICov(%) — out/coreutils-8.25/1h/setuidgid
Error for ICov(%) — out/coreutils-9.4/1h-3/hostname
Error for ICov(%) — out/coreutils-9.4/1h-3/setuidgid
Error for ICov(%) — out/coreutils-9.4/1h-2/hostname
Error for ICov(%) — out/coreutils-9.4/1h-2/setuidgid
Error for ICov(%) — out/coreutils-9.4/1h/hostname
Error for ICov(%) — out/coreutils-9.4/1h/setuidgid
Error for BCov(%) — out/coreutils-8.25/1h-3/hostname
Error for BCov(%) — out/coreutils-8.25/1h-3/setuidgid
Error for BCov(%) — out/coreutils-8.25/1h-2/hostname
Error for BCov(%) — out/coreutils-8.25/1h-2/setuidgid
Error for BCov(%) — out/coreutils-8.25/1h/hostname
Error for BCov(%) — out/coreutils-8.25/1h/setuidgid
Error for BCov(%) — out/coreutils-9.4/1h-3/hostname
Err

### Coverage according to `gcov`

In [29]:
def read_gcov_cov(util_path: Path) -> str:
    file_path = util_path / "cov.txt"
    if file_path.exists():
        with open(file_path, "r") as file:
            res = re.search(r"File '(\.\./)?\.\./src/(\w+)\.c'\nLines executed:(\d?\d\d.\d\d)% of \d+", file.read())
            if res:
                return res.group(3)
    else:
        print(f"Error for {util_path}")
        return None

add_entry_for_utils("gcov_cov", read_gcov_cov)

Error for out/coreutils-8.25/1h-3/hostname
Error for out/coreutils-8.25/1h-3/setuidgid
Error for out/coreutils-8.25/1h-2/hostname
Error for out/coreutils-8.25/1h-2/setuidgid
Error for out/coreutils-8.25/1h/hostname
Error for out/coreutils-8.25/1h/setuidgid
Error for out/coreutils-6.10/6h-3/uniq
Error for out/coreutils-6.10/6h-3/who
Error for out/coreutils-6.10/6h-3/tsort
Error for out/coreutils-9.4/1h-3/hostname
Error for out/coreutils-9.4/1h-3/setuidgid
Error for out/coreutils-9.4/1h-3/cksum
Error for out/coreutils-9.4/1h-3/wc
Error for out/coreutils-9.4/1h-2/hostname
Error for out/coreutils-9.4/1h-2/setuidgid
Error for out/coreutils-9.4/1h-2/cksum
Error for out/coreutils-9.4/1h-2/wc
Error for out/coreutils-9.4/1h/hostname
Error for out/coreutils-9.4/1h/setuidgid
Error for out/coreutils-9.4/1h/cksum
Error for out/coreutils-9.4/1h/wc


## Plots
### Massaging `df`s together

In [30]:
dfs = []
for k, v in runs.items():
    df = v["df"]
    df = df.reset_index(names="key")
    df = df.melt(id_vars="key", var_name="util")
    df["run"] = k.name
    df["time"] = v["time"]
    df["version"] = v["version"]
    dfs.append(df)

combined_df = pd.concat(dfs)
combined_df['value'] = combined_df['value'].astype(np.float64)
combined_df = combined_df.dropna(subset=['value'])
combined_df = combined_df.reset_index(drop=True)
print(combined_df.sample(20))

                       key      util  value      run     time         version
2229              gcov_cov       tee  78.16       1h    60min  coreutils-8.25
3117              gcov_cov        tr  80.62    24h-3  1440min  coreutils-6.10
9953             klee_ICov       seq  47.63    10min    10min  coreutils-6.10
9473      num_errors (ptr)       pwd   0.00    10min    10min  coreutils-6.10
11954            klee_BCov      kill  32.44  10min-2    10min  coreutils-6.10
4432    num_errors (total)        du   0.00      24h  1440min  coreutils-6.10
5447             klee_BCov        cp  27.79     6h-2   360min  coreutils-6.10
1307              gcov_cov    uptime  87.50     1h-2    60min  coreutils-8.25
14771    num_errors (exec)     tsort   0.00       1h    60min   coreutils-9.4
7719              gcov_cov      stty  81.14     1h-2    60min  coreutils-6.10
6477   num_errors (solver)     users   0.00       6h   360min  coreutils-6.10
13777   num_errors (total)    expand   1.00     1h-2    60min   

### Plots by coverage

In [31]:
coverage_df = combined_df
versions = natsorted(coverage_df["version"].unique())
keys = natsorted(coverage_df["key"].unique())
time_categories = natsorted(coverage_df["time"].unique())

color_map = dict(zip(time_categories, sns.color_palette(n_colors=len(time_categories))))

for key in keys:
    key_df = coverage_df[coverage_df["key"] == key].drop(columns="key")
    fig, axes = plt.subplots(
        ncols=1,
        nrows=len(versions),
        figsize=(10, 5*len(versions)),
        dpi=300,
    )
    fig.suptitle(f"Empirical Cumulative Distribution Function (ECDF) for {key}", fontsize=20, y=0.99)
    for version_i, version in enumerate(versions):
        version_df = key_df[key_df["version"] == version].drop(columns="version")
        ax = axes[version_i]
        ax.set_title(version)
        for time in natsorted(version_df["time"].unique()):
            time_df = version_df[version_df["time"] == time].drop(columns="time")
            for run_i, run in enumerate(time_df["run"].unique()):
                run_df = time_df[time_df["run"] == run].drop(columns="run")
                sns.ecdfplot(
                    y="value",
                    data=run_df,
                    ax=ax,
                    color=color_map[time],
                    label=time if run_i == 0 else "_nolegend_",
                    stat="count",
                )
        ax.legend(title="Time")
    plt.tight_layout()
    Path(f"plots/{key}").mkdir(exist_ok=True, parents=True)
    plt.savefig(f"plots/{key}/ecdf.png")
    plt.close()
    print(f"Done with {key}")

### Gains by time

In [32]:
df = combined_df[combined_df["version"] == "coreutils-6.10"].drop(
    columns=["run", "version"]
)
df = df.groupby(["key", "time", "util"], as_index=False).mean()
keys = natsorted(df["key"].unique())
fig, axes = plt.subplots(nrows=len(keys), ncols=1, figsize=(5, 3 * len(keys)), dpi=300)
for i, key in enumerate(keys):
    df_keys = df[df["key"] == key]
    df_keys = df_keys.drop(columns="key")
    order = natsorted(df_keys["time"].unique())
    df_keys["time"] = pd.Categorical(df_keys["time"], categories=order, ordered=True)
    df_keys = df_keys.sort_values(["util", "time"])
    df_keys["difference"] = df_keys.groupby("util", as_index=False)["value"].diff()
    df_keys = df_keys.drop(columns=["util", "value"])
    df_keys = df_keys.groupby(["time"], as_index=False, observed=True).mean()
    df_keys["time"] = (
        df_keys["time"].shift(1).astype(str) + " - " + df_keys["time"].astype(str)
    )
    df_keys = df_keys.dropna()
    sns.barplot(data=df_keys, x="time", y="difference", ax=axes[i])
    axes[i].set_ylabel(f"average {key} gained")
plt.tight_layout()
plt.savefig(f"plots/gains_by_time.png")
plt.close()

### Plots by util

In [33]:
def paint_util(args):
    key, util, key_df = args
    util_df = key_df[key_df["util"] == util].drop(columns="util")
    util_df = util_df.sort_values(by="version")
    times = natsorted(util_df['time'].unique())
    fig, ax = plt.subplots(1, 1, figsize=(8, 6), dpi=300)
    sns.stripplot(data=util_df, ax=ax, x='time', y="value", hue="version", order=times)
    ax.set_title(f"{util}")
    ax.set_ylabel(key)
    fig.tight_layout()
    Path(f"plots/{key}/by-util").mkdir(exist_ok=True, parents=True)
    fig.savefig(f"plots/{key}/by-util/{util}.png")
    plt.close(fig)


for key in natsorted(combined_df["key"].unique()):
    key_df = combined_df[combined_df["key"] == key].drop(columns="key")
    
    utils = natsorted(key_df["util"].unique())

    with multiprocessing.Pool() as pool:
        pool.map(paint_util, [(key, util, key_df) for util in utils])
    
    print(f"Done with {key}")


Done with gcov_cov
Done with klee_BCov
Done with klee_ICov
Done with num_errors (abort)
Done with num_errors (exec)
Done with num_errors (model)
Done with num_errors (ptr)
Done with num_errors (solver)
Done with num_errors (total)
